In [49]:
from pyspark.sql import SparkSession
import os
import sys
from config import API_KEY
from datetime import datetime, timedelta
import requests
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType, DateType, DoubleType, IntegerType
import pandas as pd
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, col, rank

In [6]:
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")
print(f"PATH: {os.environ.get('PATH')}")
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-1.8'
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")

#os.environ["PYSPARK_PYTHON"] = r"C:\Users\lucas\anaconda3\python.exe"
#os.environ["PYSPARK_DRIVER_PYTHON"] = r"C:\Users\lucas\anaconda3\python.exe"

JAVA_HOME: None
PATH: c:\Users\lucas\anaconda3\envs\pyspark_env;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV\Library\mingw-w64\bin;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV\Library\usr\bin;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV\Library\bin;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV\Scripts;C:\Users\lucas\anaconda3\envs\PYSPARK_ENV\bin;C:\Users\lucas\anaconda3\condabin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\libnvvp;C:\Program Files\Rockwell Software\RSCommon;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0;C:\Windows\System32\OpenSSH;C:\Program Files (x86)\NVIDIA Corporati

In [3]:
import findspark
findspark.init()

In [20]:
spark.stop()

In [21]:
spark = SparkSession.builder \
    .appName("weatherapp") \
    .getOrCreate()
    #.config("spark.driver.memory", "4g") \
    #.config("spark.executor.memory", "4g") \
    #.config("spark.python.worker.memory", "4g") \
    #.config("spark.driver.maxResultSize", "4g") \
    #.config("spark.pyspark.python", r"C:\Users\lucas\anaconda3\python.exe") \
    #.config("spark.pyspark.driver.python", r"C:\Users\lucas\anaconda3\python.exe") \
#spark.sparkContext.addFile(r"C:\Users\lucas\anaconda3\python.exe") 

In [ ]:
url = "http://api.weatherapi.com/v1/current.json"
#http://api.weatherapi.com/v1/history.json
api_key = API_KEY
city = 'Philadelphia'
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=90)
start_date = start_date.strftime("%Y-%m-%d")
params = {
    'key': api_key,
    'q': city
}

response = requests.get(url,params=params)
data = response.json()



In [36]:
data['current']

{'last_updated_epoch': 1732400100,
 'last_updated': '2024-11-23 17:15',
 'temp_c': 10.3,
 'temp_f': 50.5,
 'is_day': 0,
 'condition': {'text': 'Partly cloudy',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
  'code': 1003},
 'wind_mph': 13.2,
 'wind_kph': 21.2,
 'wind_degree': 292,
 'wind_dir': 'WNW',
 'pressure_mb': 1008.0,
 'pressure_in': 29.75,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 48,
 'cloud': 50,
 'feelslike_c': 7.6,
 'feelslike_f': 45.8,
 'windchill_c': 8.0,
 'windchill_f': 46.4,
 'heatindex_c': 10.7,
 'heatindex_f': 51.2,
 'dewpoint_c': 4.5,
 'dewpoint_f': 40.1,
 'vis_km': 16.0,
 'vis_miles': 9.0,
 'uv': 0.0,
 'gust_mph': 17.9,
 'gust_kph': 28.8}

In [ ]:
#history
def loop_date(start_date,end_date):
    date_list = []
    while start_date <= end_date:
        date_list.append(start_date.strftime("%Y-%m-%d"))
        start_date += timedelta(days=1)
    return date_list

def get_city_attr(cities,start_date,end_date):
    url = "http://api.weatherapi.com/v1/history.json"
    api_key = API_KEY
    date_list = loop_date(start_date,end_date)
    measure_metrics = ['avgtemp_f','maxwind_mph','totalprecip_in','condition']
    spark_values_store = []
    for city in cities:
        for date in date_list:
            params = {
                'key': api_key,
                'q': city,
                'dt': date
            }
            response = requests.get(url,params=params)
            data = response.json()
            date_store = {'date':date, 'city':city}
            for metric in measure_metrics:
                date_store[metric] = data['forecast']['forecastday'][0]['day'][metric]
            spark_values_store.append(date_store)
    return spark_values_store


In [39]:
def get_city_attr(cities,unit ='F'):
    url = "http://api.weatherapi.com/v1/current.json"
    api_key = API_KEY
    measure_metrics = ['name','temp_f','humidity','condition']
    spark_values_store = []
    for city in cities:
        params = {
            'key': api_key,
            'q': city
        }
        response = requests.get(url,params=params)
        data = response.json()
        data_store = {}
        for metric in measure_metrics:
            if metric == 'name':
                data_store[metric] = data['location'][metric]
                continue
            if metric == 'condition':
                data_store[metric] = data['current'][metric]['text']
                continue
            data_store[metric] = data['current'][metric]
        if unit == 'C':
            data_store['temp_c'] = (data_store['temp_f'] - 32) * (5/9)
            del data_store['temp_f']
        spark_values_store.append(data_store)
    return spark_values_store


In [11]:
cities = ['Philadelphia','Atlanta','Denver']

In [29]:
cities = [
    "New York", "Los Angeles", "Chicago", "Houston", "Phoenix",
    "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose",
    "London", "Paris", "Berlin", "Madrid", "Rome",
    "Tokyo", "Beijing", "Mumbai", "Sydney", "Cape Town",
    "Dubai", "Singapore", "Seoul", "Moscow", "Rio de Janeiro",
    "Buenos Aires", "Cairo", "Bangkok", "Istanbul", "Toronto",
    "Mexico City", "Jakarta", "Kuala Lumpur", "Lagos", "Nairobi",
    "Johannesburg", "Melbourne", "Lima", "Bogotá", "Santiago",
    "Amsterdam", "Brussels", "Warsaw", "Vienna", "Stockholm",
    "Oslo", "Copenhagen", "Helsinki", "Prague", "Zurich"
]

In [12]:
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=90)

In [13]:
df_list = get_city_attr(cities,start_date,end_date)
    

In [40]:
weather_df = get_city_attr(cities)

In [15]:
schema = StructType([
    StructField('date', StringType(), True),
    StructField('city', StringType(), True),
    StructField('avgtemp_f', FloatType(), True),
    StructField('maxwind_mph', FloatType(), True),
    StructField('totalprecip_in', FloatType(), True)
])

In [ ]:
def clean_record(record):
    return {
        'date' : str(record['date']),
        'city' : str(record['city']),
        'avgtemp_f' : double(record['avgtemp_f']),
        'maxwind_mph' : double(record['maxwind_mph']),
        'totalprecip_in' : float(record['totalprecip_in'])
    }

In [41]:
weather_df = spark.createDataFrame(weather_df)

In [ ]:
#find the city with the highest temperature
window_spec = Window.partitionBy().orderBy(col('temp_f').desc())
df_with_rank = weather_df.withColumn('rank',rank().over(window_spec))
city_with_highest_temp = df_with_rank.filter(col('rank')==1).select('name').collect()[0][0]
print(city_with_highest_temp)


In [ ]:
def city_compare(city1,city2,df):
    if 


'Jakarta'

In [62]:
df_with_rank.show()

+-------------+--------+--------------+------+----+
|    condition|humidity|          name|temp_f|rank|
+-------------+--------+--------------+------+----+
|         Mist|      63|       Jakarta|  89.8|   1|
|Partly cloudy|      70|     Singapore|  86.5|   2|
|Partly cloudy|      66|       Bangkok|  86.4|   3|
|Partly cloudy|      79|  Kuala Lumpur|  84.6|   4|
|        Clear|      94|         Lagos|  79.3|   5|
|        Sunny|      65|        Sydney|  78.1|   6|
|        Sunny|      69|         Dubai|  77.4|   7|
|Partly cloudy|      65|     Melbourne|  77.0|   8|
|     Overcast|      44|        Mumbai|  75.4|   9|
|     Overcast|      78|  Buenos Aires|  72.0|  10|
|        Clear|      73|Rio De Janeiro|  72.0|  10|
|Partly cloudy|      21|       Phoenix|  71.1|  12|
|Partly Cloudy|      82|          Lima|  65.7|  13|
|        Clear|      47|   San Antonio|  64.9|  14|
|     Overcast|      64|       Houston|  64.4|  15|
|Partly cloudy|      60|     San Diego|  63.0|  16|
|        Cle